In [1]:
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
# from agents.dqn.agent import Agent, Config
# from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper

### Basic constants and setting up environment

In [2]:
scenario = "convergence"

simTime = 10 # seconds
stepTime = 0.01  # seconds
history_length = 100


EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "nonZeroStart": True,
    "scenario": "convergence",
    "nWifi": 50
}
print("Steps per episode:", steps_per_ep) 

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

Steps per episode: 1000


In [3]:
# config = Config(buffer_size=2e3, batch_size=64, gamma=0.99, tau=1e-3, lr=5e-4)
# config = Config(buffer_size=1.5e4*threads_no, batch_size=64, gamma=0.99, tau=1e-3, lr_actor=6e-5, lr_critic=1e-3)

In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Observation space shape: (1, 'Box(100,)')
Action space shape: (1, 'Box(1,)')


### Creating and training agent

In [5]:
# import tensorflow as tf

# class Network(QNetworkTf):
#     def _inference(self):
#         with tf.variable_scope("inference_"+self.name):
#             layer = tf.layers.dense(self.input, 128, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 128, activation=tf.nn.relu)
# #             layer = tf.layers.batch_normalization(layer)
#             layer = tf.layers.dense(layer, 64, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, 32, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 256, activation=tf.nn.relu)
# #             layer = tf.layers.dense(layer, 64, activation=tf.nn.relu)
#             layer = tf.layers.dense(layer, self.action_size)
#         return layer

In [6]:
optimizer = Optimizer("OZwyhJHyqzPZgHEpDFL1zxhyI")
  # Declare your hyper-parameters:
# actor_fc1 integer [1, 4] [2]
# actor_fc2 integer [1, 4] [2]
# actor_fc3 integer [1, 4] [2]

# critic_fc1 integer [1, 4] [2]
# critic_fc2 integer [1, 4] [2]
# critic_fc3 integer [1, 4] [2]

# params = """
# lr_actor real [1e-5, 5e-4] [6e-5] log
# lr_critic real [5e-4, 1e-3] [8e-4] log
# """
params = """
lr_actor real [1e-4, 5e-3] [1e-3] log
lr_critic real [1e-3, 5e-2] [8e-3] log
"""
optimizer.set_params(params)

teacher = Teacher(env, 1)

while True:
    suggestion = optimizer.get_suggestion()
    
    actor_l = [32, 16, 8]        # [2**(suggestion["actor_fc1"]+5), 2**(suggestion["actor_fc2"]+4), 2**(suggestion["actor_fc3"]+3)]
    critic_l = [32, 16, 8]      # [2**(suggestion["critic_fc1"]+5), 2**(suggestion["critic_fc2"]+4), 2**(suggestion["critic_fc3"]+3)]
    
    lr_actor = suggestion["lr_actor"]
    lr_critic = suggestion["lr_critic"]
    
#     lr_actor = 3e-4
#     lr_critic = 4e-3
    
    config = Config(buffer_size=1.5e4*threads_no, batch_size=256, gamma=0.99, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic)
    
    print("Params:")
    for k, v in suggestion.params.items():
        print(f"{k}: {v}")
    
    agent = Agent(history_length, action_size=1, config=config, actor_layers = actor_l, critic_layers = critic_l)

    # Test the model
#     logger = teacher.train(agent, EPISODE_COUNT, simTime, stepTime, "Inp: collisions mb", "Rew: normalized speed", "DDPG", "Convergence", f"Actor: {actor_l}", f"Critic: {critic_l}", f"Instances: {threads_no}", "LSTM",
#                            f"Station count: {sim_args['nWifi']}",
#                           **config.__dict__)
    logger = teacher.train(agent, EPISODE_COUNT, simTime, stepTime, "BEB", f"Station count: {sim_args['nWifi']}")
    
    # Report the score back
    suggestion.report_score("last_speed", logger.last_speed)
    del agent


SCRIPT_RUNNING = False

Params:
lr_actor: 0.001
lr_critic: 0.008000000000000004


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/5e535bb6474643a889ec2d38f9ee67c5

 26%|███████████████▊                                             | 260/1000 [01:16<03:37,  3.41it/s, mb_sent=85.73 Mb]

------- STARTED TRAINING -------


100%|███████████████████████████████████████████████████████████| 1000/1000 [08:07<00:00,  2.05it/s, mb_sent=297.51 Mb]


Sent 297.51 Mb/s.	Mean speed: 29.75 Mb/s	Episode 1/15 finished



 78%|██████████████████████████████████████████████▉             | 783/1000 [06:10<01:42,  2.11it/s, mb_sent=239.26 Mb]


KeyboardInterrupt: 

In [ ]:
config.__dict__